In [1]:
require(tidyverse)
require(rstan)
require(ggplot2)
require(mvtnorm)
rstan_options(auto_write = TRUE)
require(bayesplot)
missing_val=-100

Loading required package: tidyverse

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.2.1     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.4
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: rstan

Loading required package: StanHeaders

rstan (Version 2.19.3, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)


Attaching package: ‘rstan’


The following object is masked from ‘package:tidyr’:

    extract


Loading required package: mvtnorm

Loading required package: bayesplot

This is bayesplot version 1.7.1

- Online documentation and v

In [2]:
path = file.path('/nfs/turbo/umms-cgillies/cgillies/RACE_CAPS/analysis', '/20200728_fixed_prior_no_missing')
dir.create(path)
setwd(path)

In [3]:
df_nmr = read_csv('/nfs/turbo/umms-cgillies/cgillies/RACE_CAPS/11_20_2019_even_odds_interaction_prior_1/nmr_metabolites_scaled_matrix.csv')
metabolites = colnames(df_nmr)[1:27]

Parsed with column specification:
cols(
  .default = col_double()
)

See spec(...) for full column specifications.



In [4]:
#TARGET_MISSING_RATE=0.3
get_alpha_beta_for_target_rate = function(df,metabolites,TARGET_MISSING_RATE) {
    missing_rate = df_nmr[,metabolites] %>% summarise_all(function(x) { mean(x == min(x))  })
    missing_rate_numeric = missing_rate %>% t() %>% as.numeric
    mu = mean( missing_rate_numeric )
    sigma_2 = var(missing_rate_numeric)
    RATE = TARGET_MISSING_RATE/mu
    mu = RATE * mu
    sigma_2 = RATE * sigma_2
    mu
    sigma_2
    nu = mu * (1 - mu) / sigma_2 - 1
    alpha = mu * nu
    beta = (1 - mu) * nu
    list(alpha=alpha,beta=beta,mu=mu,sigma_2=sigma_2)
}

get_alpha_beta_for_target_rate(df_nmr,metabolites,0.3)

$alpha
[1] 2.435085

$beta
[1] 5.681866

$mu
[1] 0.3

$sigma_2
[1] 0.02303402

In [5]:
died = df_nmr$died_90_day
sigma_0 = cov(df_nmr[died == 0,c(1:27)])
mu_0 = colMeans(df_nmr[died == 0,c(1:27)]) 
sigma_1 = cov(df_nmr[died == 1,c(1:27)])
mu_1 = colMeans(df_nmr[died == 1,c(1:27)]) 

### Generation code

In [6]:
make_df = function(n_0,mu_0,sigma_0,n_1,mu_1,sigma_1,frac_sig=0.5,censor=TRUE,max_missing=0.6,alpha=-1,beta=-1) {
    diffs = abs( mu_1 - mu_0) / sqrt( (diag(sigma_0)^2)/n_0 + (diag(sigma_1)^2)/n_1  )
    #print(diffs)
    m = length(diffs)
    diffs = diffs/sum(diffs)
    #print(diffs)
    ind = seq(1,m)
    set_non_zero = sample(ind,size=ceiling(frac_sig * m),prob=diffs)
    set_zero = setdiff(ind,set_non_zero)
    truth = rep(FALSE,m)
    truth[set_non_zero] = TRUE
    mu_0[set_zero] = 0
    mu_1[set_zero] = 0
    X_0 = rmvnorm(n = n_0, mean = mu_0, sigma = sigma_0) 
    X_1 = rmvnorm(n = n_1, mean = mu_1, sigma = sigma_1)
    df_sample = data.frame(rbind(X_0,X_1))
    df_sample$died_90_day = c( rep(0,n_0),rep(1,n_1)  )
    
    df_censored = df_sample
    df_naive_impute = df_sample
    n_var = length(mu_0)
    thresholds = rep(0,n_var)
    est.thresholds = rep(0,n_var)
    est.naive_impute = rep(0,n_var)
    missing_rates=rep(0,n_var)
    if(censor == TRUE) {
        
        missing_rates = rbeta(n_var,shape1 = alpha, shape2 = beta)
        # only allow a missing rate up to a certain level
        missing_rates = sapply(missing_rates, function(x) { min(max_missing,x) })
        
        #missing_rates = rep(MISSING_RATE, n_var)
        
        for(i in 1:n_var) {
            thresholds[i] = quantile( df_censored[, i], missing_rates[i] )
            df_censored[ (df_censored[, i] < thresholds[i]), i ] = NA
            est.thresholds[i] = min( df_censored[ , i ], na.rm = T)
            est.naive_impute[i] = log( min( exp(df_censored[ , i ]), na.rm = T) / 2 )
            df_naive_impute[ (df_naive_impute[, i] < thresholds[i]), i ] = est.thresholds[i]
        }
    }
    
    
    list(df_sample=df_sample,truth=truth, df_censored=df_censored, df_naive_impute=df_naive_impute, missing_rates=missing_rates, thresholds=thresholds, est.thresholds=est.thresholds, est.naive_impute=est.naive_impute)
}

### Stage 1 Imputation model

In [7]:
stage_1_impute = "
functions {
    real normal_ub_rng(real mu, real sigma, real ub) {
      real p_ub = normal_cdf(ub, mu, sigma);
      real u = uniform_rng(0, p_ub);
      real y = mu + sigma * inv_Phi(u);
      return y;
    }
}
data {
    int<lower=0> N;
    int<lower=0> N_var;
    int<lower=0> N_miss;
    int<lower=0> N_use_impute;

    matrix[ N, N_var ] x_naive_impute;
    matrix[ N, N_var ] x_is_missing;

    //contains the indicies of x_naive_imp to use for imputation
    int x_use_impute[N_use_impute, N_var ];

    //Threshold for imputation
    //Separate threshold per metabolite
    vector[N_var] threshold;
    

    // This vector contains the missing row index, column index
    int x_missing[ N_miss, 2 ];
    
}
parameters {

    //betas for each imputed value
    matrix[ N_use_impute, N_var ] beta_impute_raw;
    vector[ N_var ] alpha_impute;
    vector<lower=0>[ N_var ] sigma_impute;
    real<lower=0> sigma_beta_impute_x;

}
transformed parameters {
    matrix[ N_use_impute, N_var ] beta_impute;
    for(j in 1:N_var) {
        beta_impute[,j] = beta_impute_raw[,j] * sigma_beta_impute_x;
    }
}
model {
    sigma_beta_impute_x ~ std_normal();
    // FORGOT TO SET PRIOR ON alpha_x_pred and sigma_impute

    for(j in 1:N_var) {
        real y[N];
        vector[N_use_impute] beta_x_pred;
        int x_use_ind[N_use_impute];
        matrix[N, N_use_impute] x_pred_use;
        real alpha_x_pred;
        real sigma_x_pred;
        //predicted target
        vector[N] xj_pred;
        //target
        vector[N] xj;
        int observed[N];
        
        //ADD TO LIKELIHOOD
        beta_impute_raw[, j] ~ std_normal();
    
        //ASSIGN VALS
        xj = x_naive_impute[, j];
        //dont make a prediction using itself
        //this should be copying the values according to ref
        beta_x_pred = beta_impute[, j];
        alpha_x_pred = alpha_impute[j];
        x_use_ind = x_use_impute[, j];
        sigma_x_pred = sigma_impute[j];
        x_pred_use = x_naive_impute[, x_use_ind];
        
        //Make pred
        xj_pred = x_pred_use * beta_x_pred + alpha_x_pred;
        
        
        //observed = (x_is_missing[, j] == 0);
        
        // target +=  normal_lpdf(xj | xj_pred, sigma_x_pred);

        for(i in 1:N) {
            real mu_x_pred;
            mu_x_pred = xj_pred[i];
            if( x_is_missing[i,j] == 0) {
                // if observed use pdf for observed value
                target +=  normal_lpdf(xj[i] | mu_x_pred, sigma_x_pred);
            } else {
                // if censored, use normal CDF
                target +=  normal_lcdf(threshold[j] | mu_x_pred, sigma_x_pred);
            }
        }
    }
}
generated quantities {
    real x_impute_mean[ N_miss ];
    real x_impute[ N_miss ];

    for(i in 1:N_miss) {
        //These correspond to the rows and columns in the original data frame
        int metab_row;
        int metab_col;
        row_vector[N_use_impute] x_pred_use;
        vector[N_use_impute] beta_x_pred;
        real alpha_x_pred;
        int x_use_ind[N_use_impute];
        real mu_x;
        real sd_x;
        real ub_x;

        metab_row = x_missing[i,1];
        metab_col = x_missing[i,2];
        x_use_ind = x_use_impute[, metab_col ];
        beta_x_pred = beta_impute[, metab_col ];
        alpha_x_pred = alpha_impute[ metab_col ];
        x_pred_use = x_naive_impute[metab_row, x_use_ind];

        mu_x = x_pred_use * beta_x_pred + alpha_x_pred;
        sd_x =  sigma_impute[ metab_col ];
        ub_x = threshold[ metab_col ];
        x_impute_mean[ i ] = mu_x;
        x_impute[i] = normal_ub_rng(mu_x, sd_x, ub_x);
    }
}

"

In [8]:
stage_1_impute_code_file = 'stage_1_impute.stan'
write_lines(x = stage_1_impute, path = stage_1_impute_code_file)
stage_1_impute_model = stan_model(file = stage_1_impute_code_file, verbose = FALSE)

In [9]:
stage_1_impute_get_init_vals = function(data) {
    n_var = data$N_var
    N_use_impute = data$N_use_impute
    # matrix[ N_use_impute, N_var ] beta_impute_raw;
    beta_impute_raw = matrix( rep(0, data$N_use_impute * n_var), ncol=n_var)
    alpha_impute = rep(0,n_var)
    x_use_impute = data$x_use_impute
    sigma_impute = rep(1, n_var)
    
    for(i in 1:n_var) {
        x_imp_vals_use = data$x_naive_impute[, x_use_impute[,i]   ]
        y_out_imp = data$x_naive_impute[,i]
        m_out = glm(y_out_imp ~ x_imp_vals_use,family="gaussian")
        coefs_imp = summary(m_out)$coef
        alpha_impute[i] = coefs_imp[1,1]
        for(k in 1:N_use_impute) {
            beta_impute_raw[k,i] = coefs_imp[1 + k,1]
        }
        sigma_impute[i] = sum(m_out$residuals^2) / m_out$df.residual
    }
    
    sigma_beta_impute_x = sqrt( sum( beta_impute_raw^2 ) / (prod(dim(beta_impute_raw)) - 1) ) 
    
    inits = list(
        sigma_impute = sigma_impute,
        sigma_beta_impute_x = sigma_beta_impute_x,
        alpha_impute = alpha_impute,
        beta_impute_raw = beta_impute_raw / sigma_beta_impute_x
    )
    
    inits
    
}

In [10]:
stage_1_impute_make_data = function(x_censored, y, threshold=NULL,
                                         N_use_impute=8) {
    
    if(is.null(threshold)) {
        threshold = apply(x_censored,MARGIN=2,FUN=function(x) { min(x,na.rm = T)})
    }
    
    naive_impute_val = log( exp(threshold) / 2)
    
    n = dim(x_censored)[1]
    N_var = dim(x_censored)[2]
    N_miss = sum(is.na(x_censored))
    
    x_naive_impute = x_censored %>% select_all()
    x_is_missing = apply( is.na(x_censored), FUN=as.numeric, MARGIN=c(1,2))             
    for(j in 1:N_var) {
        x_naive_impute[is.na(x_naive_impute[,j]),j] = naive_impute_val[j]
    }            
    # create a matrix that contains the N_use_impute indicies of the most correlated
    # variables with the metabolite of interest
    # each column contains a metabolite corresponding with x_censored
    # each row corresponds to the index of the metabolite most correlated with it up to N_use_impute
    # rows
    var_cors = cor(x_naive_impute,use = "complete")
    diag(var_cors) = 0
    get_inds_to_use_per_metabolite = function(x) { which( rank(-abs(x)) <= N_use_impute)   }
    x_use_impute = apply(var_cors,MARGIN=1,FUN=get_inds_to_use_per_metabolite )

    x_censored[is.na(x_censored)] = missing_val

    x_censored = as.matrix(x_censored)
    x_missing = matrix(nrow = N_miss, ncol = 2)
    x_missing_init = rep(0,N_miss)
    x_missing_init_raw = rep(0,N_miss)
    
    # make a table were the first column is the row number in x_censored
    # second column contains the metabolite column number in x_censored
    col1 = c()
    col2 = c()
    for(j in 1:dim(x_censored)[2]) {
        missing_in_col = which(x_censored[,j] == missing_val)
        col1 = c(col1, missing_in_col)
        col2 = c(col2, rep(j,length(missing_in_col)))
    }
    x_missing[,1] = col1
    x_missing[,2] = col2
    
    if( N_miss > 0 ) {
        # make a vector with naive imputation values
        for(i in 1:dim(x_missing)[1]) {
            row = x_missing[i,1]
            col = x_missing[i,2]
            x_missing_init[i] = x_naive_impute[row,col]
            x_missing_init_raw[i] = x_missing_init[i] - threshold[col]
        }
    }

    data = list(
            N = n,
            N_miss = N_miss,
            N_var = N_var,
            x_raw = x_censored,
            x_naive_impute = as.matrix(x_naive_impute),
            N_use_impute = N_use_impute,
            x_use_impute = x_use_impute,
            x_missing = x_missing,
            x_missing_init = x_missing_init,
            x_missing_init_raw = x_missing_init_raw,
            x_is_missing = x_is_missing,
            threshold = threshold,
            y=y
        )

    #str(data)
    #mean(x_censored == missing_val)
    #data$N_miss / (data$N * data$N_var)

    data   
}


In [11]:
replicate_init = function(init_data,chains=4) {
    init = list()
    for(i in 1:chains){
        init[paste0(i)] = list(init_data)
    }
    init
} 

In [12]:
set.seed(0)
n_0=100
n_1=100
beta_params = get_alpha_beta_for_target_rate(df_nmr,metabolites,TARGET_MISSING_RATE = 0.3)
#out = make_df(n_0,mu_0,sigma_0,n_1,mu_1,sigma_1,frac_sig=0.4,censor=TRUE,
#                  max_missing = 0.4, alpha = beta_params$alpha, beta=beta_params$beta)

out = make_df(n_0,mu_0,sigma_0,n_1,mu_1,sigma_1,frac_sig=0.4,censor=FALSE,
                  max_missing = 0.4, alpha = beta_params$alpha, beta=beta_params$beta)


write_rds(out,'debug_df.rds')

x_censored = out$df_censored[,metabolites]
y=out$df_censored$died_90_day
sum(is.na(x_censored))

stage_1_data = stage_1_impute_make_data(x_censored, y, N_use_impute=8)
stage_1_init_data = replicate_init(
        stage_1_impute_get_init_vals(stage_1_data), chains=1
    )
str(stage_1_data)

[1] 0

List of 13
 $ N                 : int 200
 $ N_miss            : int 0
 $ N_var             : int 27
 $ x_raw             : num [1:200, 1:27] -0.2539 0.1963 0.0995 1.0577 -0.5308 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ x_naive_impute    : num [1:200, 1:27] -0.2539 0.1963 0.0995 1.0577 -0.5308 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ N_use_impute      : num 8
 $ x_use_impute      : int [1:8, 1:27] 3 15 17 18 19 22 26 27 1 4 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ x_missing         : int[0 , 1:2] 
 $ x_missing_init    : num(0) 
 $ x_missing_init_raw: num(0) 
 $ x_is_missing      : num [1:200, 1:27] 0 0 0 0 0 0 0 0 0 0 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..

In [13]:
stage_1_fit = sampling(stage_1_impute_model,  data = stage_1_data, init=stage_1_init_data, cores=1, chains=1, iter = 2000, control=list(adapt_delta=0.8))



SAMPLING FOR MODEL 'stage_1_impute' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 68.64 seconds (Warm-up)
Chain 1:                46.38 seconds (Sampling)
Chain 1:                115.02 se

In [14]:
df_sum = data.frame(summary(stage_1_fit)$summary) 
df_sum$PARAM = rownames(summary(stage_1_fit)$summary)
df_sum %>% arrange(n_eff) %>% head(10)
df_sum %>% arrange(-abs(Rhat)) %>% head(10)

,mean,se_mean,sd,X2.5.,X25.,X50.,X75.,X97.5.,n_eff,Rhat,PARAM
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,-5512.6520737,1.1956927349,16.06832048,-5.543797e+03,-5.523679e+03,-5513.1013050,-5501.2417105,-5480.7377332,180.5934,0.9995295,lp__
2,0.1972392,0.0006506298,0.01056270,1.780939e-01,1.895429e-01,0.1964747,0.2043881,0.2188030,263.5613,0.9991405,sigma_beta_impute_x
3,3.6329327,0.0134496092,0.27568563,3.097374e+00,3.445525e+00,3.6250536,3.8110687,4.1945714,420.1544,0.9991785,"beta_impute_raw[7,16]"
4,4.3375261,0.0148969168,0.31698955,3.720797e+00,4.117323e+00,4.3296303,4.5447819,4.9611346,452.7903,0.9990645,"beta_impute_raw[7,25]"
5,3.3370878,0.0127093304,0.28440287,2.791659e+00,3.141564e+00,3.3299810,3.5263381,3.9184802,500.7519,0.9997764,"beta_impute_raw[8,17]"
6,3.4715000,0.0103898173,0.31538750,2.843631e+00,3.245802e+00,3.4727668,3.6934118,4.0503065,921.4529,0.9993288,"beta_impute_raw[5,27]"
7,1.4982265,0.0069840063,0.22234685,1.061910e+00,1.343338e+00,1.5059335,1.6490110,1.9177824,1013.5676,0.9995534,"beta_impute_raw[4,4]"
8,0.5832399,0.0092835255,0.30322833,2.510396e-02,3.715368e-01,0.5748025,0.7738567,1.2013517,1066.8754,0.9990317,"beta_impute_raw[6,4]"
9,0.1147143,0.0017639371,0.05902539,5.266292e-03,7.296193e-02,0.1127369,0.1549204,0.2334397,1119.7256,0.9990989,"beta_impute[6,4]"


,mean,se_mean,sd,X2.5.,X25.,X50.,X75.,X97.5.,n_eff,Rhat,PARAM
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,0.51640867,0.011842587,0.41118668,-0.30030119,0.25400458,0.48908695,0.787742967,1.36391972,1205.549,1.003561,"beta_impute_raw[3,21]"
2,0.23161804,0.001574323,0.07256845,0.08728896,0.18406437,0.23234613,0.278254824,0.37151915,2124.750,1.003547,"beta_impute[1,9]"
3,0.10170174,0.002306241,0.08079381,-0.05972333,0.04961109,0.09612933,0.154734287,0.26551895,1227.289,1.003540,"beta_impute[3,21]"
4,-0.22552211,0.009534655,0.38590639,-0.98936137,-0.48054038,-0.23625986,0.036739931,0.49465571,1638.151,1.003409,"beta_impute_raw[6,21]"
5,-0.04452559,0.001859412,0.07585322,-0.19545827,-0.09451529,-0.04647340,0.007485014,0.09729545,1664.167,1.003375,"beta_impute[6,21]"
6,1.17621741,0.008417771,0.36959853,0.44055025,0.92927941,1.17041269,1.418843823,1.93140420,1927.819,1.002851,"beta_impute_raw[1,9]"
7,-0.11382482,0.006487158,0.28030502,-0.68483057,-0.29978539,-0.10539689,0.056003849,0.42525222,1867.037,1.002350,"beta_impute_raw[4,16]"
8,-0.02235305,0.001294869,0.05498725,-0.13486865,-0.05828562,-0.02075392,0.011042828,0.08294994,1803.319,1.002322,"beta_impute[4,16]"
9,0.79403343,0.008869983,0.38462335,0.06309948,0.53065739,0.77645881,1.070965202,1.53488170,1880.294,1.002314,"beta_impute_raw[5,24]"


### Stage 2 regression code

In [15]:
stage_2_regress = "
data {
    int<lower=0> N;
    int<lower=0> N_var;
    int<lower=0> N_miss;

    matrix[ N, N_var ] x_raw;

    int<lower=0,upper=1> y[N];

    //Threshold for imputation
    //Separate threshold per metabolite
    vector[N_var] threshold;
    
    // This vector contains the missing row index, column index
    int x_missing[ N_miss, 2 ];
    real x_missing_mean[ N_miss ];
    real x_missing_sd[ N_miss ];
    
}
parameters {
    // Needed for imputation
    vector<upper=0>[ N_miss ] x_impute_raw;
    
    //regression on y per metabolite in x_raw columns
    vector[N_var] beta_x;
    vector[N_var] alpha_x;

    real<lower=0> sigma_beta_x;
    real<lower=1> nu_x;

}
transformed parameters {
    // https://mc-stan.org/docs/2_18/stan-users-guide/vectors-with-varying-bounds.html
    vector<upper=0>[ N_miss ] x_impute;
    x_impute = x_impute_raw;
    for(i in 1:N_miss) {
        int metab_row = x_missing[i,1];
        int metab_col = x_missing[i,2];
        x_impute[i] = x_impute[i] + threshold[metab_col];
    }

}
model {
    matrix[N, N_var] x_merge;
    //copy matrix
    x_merge = x_raw;
    
    //fill in the values we are going to impute
    for(i in 1:N_miss) {
        int metab_row = x_missing[i,1];
        int metab_col = x_missing[i,2];
        x_merge[metab_row,metab_col] = x_impute[ i ];
    }
    
    x_impute ~ normal(x_missing_mean, x_missing_sd);
    
    sigma_beta_x ~ cauchy(0,1);
    nu_x ~ gamma(2,0.1);
    //FIX PRIOR FOR BETA
    beta_x ~  student_t(100,0,5);
    alpha_x ~ normal(0,5);
    
    for(j in 1:N_var) {
        vector[N] x_col = x_merge[ , j];
        real beta = beta_x[j];
        real alpha = alpha_x[j];
        y ~ bernoulli_logit(alpha + beta * x_col);
    }
}

"

In [16]:
stage_2_regress_code_file = 'stage_2_regress.stan'
write_lines(x = stage_2_regress, path = stage_2_regress_code_file)
stage_2_regress_model = stan_model(file = stage_2_regress_code_file, verbose = FALSE)

DIAGNOSTIC(S) FROM PARSER:
Info:
Left-hand side of sampling statement (~) may contain a non-linear transform of a parameter or local variable.
If it does, you need to include a target += statement with the log absolute determinant of the Jacobian of the transform.
Left-hand-side of sampling statement:
    x_impute ~ normal(...)



In [17]:
###
# This function formats the data for
###

default_impute = function(x_censored, x_missing) {
    N_miss = dim(x_missing)[1]
    #print(N_miss)
    x_missing_mean = rep(0,N_miss)
    x_missing_sd = rep(1,N_miss)
    
    list(
         x_missing_mean=x_missing_mean,
         x_missing_sd=x_missing_sd
        )
}

stage_2_regress_make_data = function(x_censored, y, IMPUTE_FUNC=default_impute, threshold=NULL, ...) {
    
    if(is.null(threshold)) {
        threshold = apply(x_censored,MARGIN=2,FUN=function(x) { min(x,na.rm = T)})
    }
    
    naive_impute_val = log( exp(threshold) / 2)
    
    n = dim(x_censored)[1]
    N_var = dim(x_censored)[2]
    N_miss = sum(is.na(x_censored))
    
    x_naive_impute = x_censored %>% select_all()
                 
    for(j in 1:N_var) {
        x_naive_impute[is.na(x_naive_impute[,j]),j] = naive_impute_val[j]
    }            


    x_censored[is.na(x_censored)] = missing_val

    x_censored = as.matrix(x_censored)
    x_missing = matrix(nrow = N_miss, ncol = 2)
    x_missing_init = rep(0,N_miss)
    x_missing_init_raw = rep(0,N_miss)
    x_missing_mean = rep(0,N_miss)
    x_missing_sd = rep(1,N_miss)
    
    # make a table were the first column is the row number in x_censored
    # second column contains the metabolite column number in x_censored
    col1 = c()
    col2 = c()
    for(j in 1:dim(x_censored)[2]) {
        missing_in_col = which(x_censored[,j] == missing_val)
        col1 = c(col1, missing_in_col)
        col2 = c(col2, rep(j,length(missing_in_col)))
    }
    x_missing[,1] = col1
    x_missing[,2] = col2
    
    # make a vector with naive imputation values
    if( N_miss > 0 ) {
        for(i in 1:dim(x_missing)[1]) {
            row = x_missing[i,1]
            col = x_missing[i,2]
            x_missing_init[i] = x_naive_impute[row,col]
            x_missing_init_raw[i] = x_missing_init[i] - threshold[col]
        }
    
        mean_sds = IMPUTE_FUNC(x_censored,x_missing,...)
        x_missing_mean = mean_sds$x_missing_mean
        x_missing_sd = mean_sds$x_missing_sd
    }
    
    data = list(
            N = n,
            N_miss = N_miss,
            N_var = N_var,
            x_raw = x_censored,
            x_naive_impute = as.matrix(x_naive_impute),
            x_missing = x_missing,
            x_missing_init = x_missing_init,
            x_missing_init_raw = x_missing_init_raw,
            x_missing_mean=x_missing_mean,
            x_missing_sd=x_missing_sd,
            threshold = threshold,
            y=y
        )

    #str(data)
    #mean(x_censored == missing_val)
    #data$N_miss / (data$N * data$N_var)

    data   
}


In [18]:
stage_1_impute = function(x_censored, x_missing, stage_1_post=NULL) {
    N_miss = dim(x_missing)[1]
    print(N_miss)
    x_missing_mean = apply( stage_1_post$x_impute_mean, FUN=mean, MARGIN=2 )
    x_missing_sd = apply( stage_1_post$x_impute, FUN=sd, MARGIN=2 )
            
    list(
         x_missing_mean=x_missing_mean,
         x_missing_sd=x_missing_sd
        )
}

fancy_impute = function(x_censored, x_missing) {
    N_miss = dim(x_missing)[1]
    #print(N_miss)
    x_missing_mean = rep(0,N_miss)
    x_missing_sd = rep(1,N_miss)
    
    
    x_obs = out$df_sample[,metabolites]
    for(i in 1:N_miss) {
        row = x_missing[i,1]
        col = x_missing[i,2]
        x_missing_mean[i] = x_obs[row,col]
        x_missing_sd[i] = 0.001
    }
            
    list(
         x_missing_mean=x_missing_mean,
         x_missing_sd=x_missing_sd
        )
}

stage_1_post = extract(stage_1_fit)
stage_2_data = stage_2_regress_make_data(x_censored, y, IMPUTE_FUNC = stage_1_impute, stage_1_post=stage_1_post)
#stage_2_init = list(
#    x_impute_raw = stage_2_data$x_missing_init_raw
#)


stage_2_init = replicate_init(
    list(
        x_impute_raw = stage_2_data$x_missing_init_raw
    ), chains=4
)


In [19]:
str(stage_2_data)
str(stage_2_init)

List of 12
 $ N                 : int 200
 $ N_miss            : int 0
 $ N_var             : int 27
 $ x_raw             : num [1:200, 1:27] -0.2539 0.1963 0.0995 1.0577 -0.5308 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ x_naive_impute    : num [1:200, 1:27] -0.2539 0.1963 0.0995 1.0577 -0.5308 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ x_missing         : int[0 , 1:2] 
 $ x_missing_init    : num(0) 
 $ x_missing_init_raw: num(0) 
 $ x_missing_mean    : num(0) 
 $ x_missing_sd      : num(0) 
 $ threshold         : Named num [1:27] -2.4 -2.11 -3.15 -2.4 -2.32 ...
  ..- attr(*, "names")= chr [1:27] "hydroxybutyrate" "oxoisocaproate" "X3.hydoxybutyrate" "alanine" ...
 $ y                 : num [1:200] 0 0 0 0 0 0 0 0 0 0 ...
List of 4
 $ 1:List of 1
  ..$ x_impute_raw: num(0) 


In [20]:
#stage_2_data$threshold
sum(apply(x_censored,FUN=function(x) { sum(is.na(x))},MARGIN=2))
x_censored[ stage_2_data$x_missing %>% head ]
out$df_sample[ stage_2_data$x_missing %>% head ]
stage_2_data$x_missing_mean[1:5]

[1] 0

numeric(0)

numeric(0)

[1] NA NA NA NA NA

In [21]:
stage_2_fit = sampling(stage_2_regress_model,  data = stage_2_data, init=stage_2_init, cores=4, chains=4, iter = 2000, control=list(adapt_delta=0.8))


In [22]:
df_sum = data.frame(summary(stage_2_fit)$summary) 
df_sum$PARAM = rownames(summary(stage_2_fit)$summary)
df_sum %>% arrange(n_eff)

mean,se_mean,sd,X2.5.,X25.,X50.,X75.,X97.5.,n_eff,Rhat,PARAM
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
-3.704433e+03,0.126548683,5.2651370,-3.715826e+03,-3.707868e+03,-3.704111e+03,-3.700691e+03,-3695.0958534,1731.028,1.0037261,lp__
4.398881e+00,0.568729674,26.7871605,4.253737e-02,3.983861e-01,9.697964e-01,2.282182e+00,21.9402816,2218.409,1.0021943,sigma_beta_x
7.688324e-01,0.002496080,0.1668825,4.427219e-01,6.552435e-01,7.665002e-01,8.798266e-01,1.0973002,4469.970,0.9996718,beta_x[20]
7.018468e-01,0.002530156,0.1708062,3.732987e-01,5.837454e-01,6.975288e-01,8.196220e-01,1.0337180,4557.351,1.0009033,beta_x[5]
5.077118e-02,0.002015962,0.1362832,-2.172133e-01,-4.037707e-02,4.952101e-02,1.432931e-01,0.3147412,4570.041,0.9995925,beta_x[13]
1.503258e-03,0.002100166,0.1436815,-2.919383e-01,-9.529839e-02,2.714746e-03,9.751984e-02,0.2794762,4680.523,0.9995219,alpha_x[13]
2.415260e-02,0.002097637,0.1440358,-2.578956e-01,-7.271025e-02,2.466211e-02,1.219765e-01,0.3023593,4714.984,1.0003142,beta_x[19]
7.178767e-01,0.002417506,0.1661973,4.090116e-01,6.037847e-01,7.158628e-01,8.277593e-01,1.0572227,4726.209,0.9997213,beta_x[24]
9.633555e-03,0.002008788,0.1394824,-2.600726e-01,-8.736423e-02,9.114565e-03,1.024588e-01,0.2864574,4821.373,0.9993997,alpha_x[27]


### Run both models

In [23]:
make_summary_table_beta = function(fit,metabolites) {
    df_summary = data.frame(summary(fit)$summary)
    post = rstan::extract(fit)
    #compute prob > 0 or prob < 0
    n_sample = dim(post$beta_x)[1]
    g0 = colSums( (post$beta_x > 0) ) / n_sample 
    l0 = colSums( (post$beta_x < 0) ) / n_sample
    df_prob = data.frame(G0=g0,L0=l0) %>% rowwise() %>% mutate(Max=max(G0,L0))
    beta_names = c()
    for(i in seq(0,dim(post$beta_x)[2])) {
        beta_names=c(beta_names,paste0('beta_x[',i,']'))
    }
    df_summary_beta = df_summary %>% mutate(Param = rownames(.)) %>% 
    filter(Param %in% beta_names) %>% cbind(Metabolite = metabolites, P_GT_LT_0=df_prob$Max) %>%
    mutate(Z=mean/sd) %>% arrange(-P_GT_LT_0, -abs(Z)) %>% select(Metabolite,Param,mean,sd,X2.5.,X50.,X97.5.,Rhat,Z,P_GT_LT_0)
    return(df_summary_beta)   
}

In [24]:
run_bayes_model = function(df_censored,metabolites,cores=4,chains=4,iter=2000, 
                           adapt_delta=0.8,
                           max_treedepth=10) {
    x_censored = df_censored[, metabolites]
    y = df_censored$died_90_day
    
    stage_1_data = stage_1_impute_make_data(x_censored, y, N_use_impute=8)
    stage_1_init_data = replicate_init(
        stage_1_impute_get_init_vals(stage_1_data), chains=4
    )
    
    control = list(adapt_delta=adapt_delta,max_treedepth=max_treedepth)
    print("CONTROL")
    print(control)
    
    N_miss = stage_1_data$N_miss
    stage_1_fit = NULL
    stage_1_post = NULL
    if(N_miss > 0) {
        print("IMPUTING")
        stage_1_fit = sampling(stage_1_impute_model,  data = stage_1_data, init=stage_1_init_data, cores=cores, chains=chains, iter = iter,
                             control = control )

        stage_1_post = extract(stage_1_fit)
    } else {
        print("NO MISING DATA SKIPPING IMPUTATION")
    }
    
    stage_2_data = stage_2_regress_make_data(x_censored, y, IMPUTE_FUNC = stage_1_impute, stage_1_post=stage_1_post)
    stage_2_init = list(
        x_impute_raw = stage_2_data$x_missing_init_raw
    )
    
    stage_2_init = replicate_init(
        list(
            x_impute_raw = stage_2_data$x_missing_init_raw
        ), chains=4
    )
    
    
    stage_2_fit = sampling(stage_2_regress_model,  data = stage_2_data, init=stage_2_init, cores=cores, chains=chains, iter = iter, control = control )

    
    sum_table = make_summary_table_beta(stage_2_fit,metabolites)
    rownames(sum_table) = sum_table$Metabolite
    list( sum_table=sum_table, stage_1_fit=stage_1_fit, stage_2_fit=stage_2_fit)
}

In [25]:
make_summary_table_beta(stage_2_fit,metabolites)

Metabolite,Param,mean,sd,X2.5.,X50.,X97.5.,Rhat,Z,P_GT_LT_0
<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
oacetylcarnitine,beta_x[20],0.768832420,0.1668825,0.44272190,0.766500183,1.0973002,0.9996718,4.60702915,1.00000
propyleneglycol,beta_x[24],0.717876699,0.1661973,0.40901157,0.715862815,1.0572227,0.9997213,4.31942474,1.00000
betaine,beta_x[5],0.701846839,0.1708062,0.37329868,0.697528835,1.0337180,1.0009033,4.10902493,1.00000
glutamine,beta_x[12],0.619051182,0.1536771,0.32347413,0.618062540,0.9235908,1.0010834,4.02825788,1.00000
proline,beta_x[23],0.568269937,0.1539335,0.27278322,0.564523933,0.8785540,0.9994787,3.69165813,1.00000
alanine,beta_x[4],0.482568891,0.1486897,0.19969531,0.482519970,0.7705054,0.9999467,3.24547629,1.00000
choline,beta_x[7],0.319837725,0.1507064,0.03201524,0.316497816,0.6136711,0.9992396,2.12225766,0.98300
phenylalanine,beta_x[22],0.309115229,0.1503443,0.01330903,0.310082966,0.6044853,0.9995385,2.05604838,0.97925
pyruvate,beta_x[25],0.227283922,0.1323352,-0.01974308,0.225404422,0.4999658,0.9997764,1.71748695,0.96200


In [26]:
bayes_model_res = run_bayes_model(out$df_censored,metabolites,cores=4,chains=4,iter=2000, 
                           adapt_delta=0.8,
                           max_treedepth=10)

[1] "CONTROL"
$adapt_delta
[1] 0.8

$max_treedepth
[1] 10

[1] "NO MISING DATA SKIPPING IMPUTATION"


In [27]:
apply( out$df_censored, FUN=function(x) { mean(is.na(x)) }, MARGIN=1)

[1] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [38] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [75] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[112] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[149] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[186] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [28]:
names(bayes_model_res)

[1] "sum_table"   "stage_1_fit" "stage_2_fit"

In [29]:
df_sum = data.frame(summary(bayes_model_res$stage_2_fit)$summary) 
df_sum$PARAM = rownames(summary(bayes_model_res$stage_2_fit)$summary)
df_sum %>% arrange(n_eff) %>% head(20)

,mean,se_mean,sd,X2.5.,X25.,X50.,X75.,X97.5.,n_eff,Rhat,PARAM
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,-3.704542e+03,0.132895489,5.4282372,-3.715919e+03,-3.708096e+03,-3.704151e+03,-3.700743e+03,-3695.0479427,1668.388,1.0010548,lp__
2,6.873123e+00,2.089332569,100.4493419,3.568557e-02,4.141710e-01,1.019436e+00,2.373300e+00,21.5560589,2311.421,1.0001673,sigma_beta_x
3,3.137219e-03,0.002179719,0.1471500,-2.877845e-01,-9.830369e-02,3.961657e-03,1.003747e-01,0.2927854,4557.424,0.9996118,alpha_x[26]
4,4.539346e-02,0.002035290,0.1378376,-2.184487e-01,-4.829980e-02,4.338159e-02,1.400361e-01,0.3145677,4586.515,0.9995422,beta_x[13]
5,-1.009682e-03,0.002089115,0.1418411,-2.816090e-01,-9.582763e-02,-1.883202e-05,9.497114e-02,0.2721648,4609.770,1.0009176,alpha_x[19]
6,7.172516e-01,0.002508048,0.1705533,3.918510e-01,6.013158e-01,7.144355e-01,8.268910e-01,1.0635421,4624.323,1.0004279,beta_x[24]
7,1.070027e-02,0.002041188,0.1397305,-2.665865e-01,-8.215844e-02,1.269829e-02,1.048623e-01,0.2780800,4686.151,1.0002145,alpha_x[6]
8,1.040806e-01,0.001976177,0.1353882,-1.635081e-01,1.549904e-02,1.010569e-01,1.940645e-01,0.3720205,4693.646,1.0002924,beta_x[8]
9,-1.484270e-04,0.002088339,0.1442549,-2.774385e-01,-9.883169e-02,-1.186560e-03,9.822300e-02,0.2874004,4771.545,0.9993415,alpha_x[9]


In [30]:
bayes_model_res$sum_table

,Metabolite,Param,mean,sd,X2.5.,X50.,X97.5.,Rhat,Z,P_GT_LT_0
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
oacetylcarnitine,oacetylcarnitine,beta_x[20],7.639935e-01,0.1682887,0.445874503,0.7636496407,1.1056029,0.9994261,4.5397777044,1.00000
propyleneglycol,propyleneglycol,beta_x[24],7.172516e-01,0.1705533,0.391851026,0.7144355157,1.0635421,1.0004279,4.2054405633,1.00000
betaine,betaine,beta_x[5],6.982084e-01,0.1689681,0.373031779,0.6982148531,1.0368310,0.9994654,4.1321897727,1.00000
glutamine,glutamine,beta_x[12],6.175891e-01,0.1576760,0.318108277,0.6120563050,0.9452444,0.9991715,3.9168246301,1.00000
proline,proline,beta_x[23],5.712425e-01,0.1514566,0.282371322,0.5676142400,0.8669510,0.9992120,3.7716576290,1.00000
alanine,alanine,beta_x[4],4.906579e-01,0.1538410,0.195404879,0.4892012897,0.8031562,0.9999270,3.1893831902,0.99875
choline,choline,beta_x[7],3.095820e-01,0.1470651,0.025924879,0.3089461590,0.5995421,0.9996528,2.1050677718,0.98650
phenylalanine,phenylalanine,beta_x[22],3.026457e-01,0.1535625,0.004350538,0.3014010688,0.6039362,0.9999585,1.9708309297,0.97750
carnitine,carnitine,beta_x[6],2.476627e-01,0.1464668,-0.037689827,0.2467729756,0.5363554,0.9998012,1.6909136971,0.95700


### Test Full Simulation

In [31]:
run_sim = function(n_sim=100,n_0=100,n_1=100,frac_sig=0.4,
                   metabolites=NULL,censor=TRUE,max_missing=0.4,iter=2000,sim_number=1,MISSING_RATE=NULL,alpha=NULL,beta=NULL) {
    res = data.frame(stringsAsFactors = F)
    for(i in 1:n_sim) {
        sim_data = make_df(n_0,mu_0,sigma_0,n_1,mu_1,sigma_1,frac_sig=frac_sig,censor=censor,max_missing=max_missing,alpha = alpha, beta=beta)
        df_censored = sim_data$df_censored
        df_naive_impute = sim_data$df_naive_impute
        truth = sim_data$truth
        
        out_sim = paste0("RAW_", "SIM_NUMBER_",sim_number,"_N_SAMPLE_",n_sample,"_FRAC_SIG_",frac_sig,'_MISSING_RATE_',MISSING_RATE,'_SIM_DATA','.rds' )
        print("WRITNG OUT...")
        print(out_sim)
        write_rds(sim_data, out_sim, compress = 'gz')
        
        df_eff_sim = sapply(metabolites,FUN=function( x ) { apply_logistic_sim(x,df_naive_impute) } )
        sig_raw = as.vector( df_eff_sim['Pr(>|z|)',] < 0.05)
        eff_raw = as.vector( df_eff_sim['Estimate',])
        
        sig_bon = as.vector( p.adjust( df_eff_sim['Pr(>|z|)',],method='bonferroni') < 0.05)
        eff_bon = as.vector( df_eff_sim['Estimate',])
        
        sig_bh = as.vector( p.adjust( df_eff_sim['Pr(>|z|)',],method='BH') < 0.05)
        eff_bh = as.vector( df_eff_sim['Estimate',])
        
        bayes_model_res = run_bayes_model(df_censored,metabolites,cores=4,chains=4,iter=2000, 
                           adapt_delta=adapt_delta,
                           max_treedepth=max_treedepth)
        
        
        sum_table = bayes_model_res$sum_table
        stage_1_fit = bayes_model_res$stage_1_fit
        stage_2_fit = bayes_model_res$stage_2_fit
        
        out_fit = paste0("RAW_", "SIM_NUMBER_",sim_number,"_N_SAMPLE_",n_sample,"_FRAC_SIG_",frac_sig,'_MISSING_RATE_',MISSING_RATE,'_FIT_DATA','.rds' )
        print("WRITNG OUT...")
        print(out_fit)
        write_rds(bayes_model_res, out_fit, compress = 'gz')

        sig_bayes = as.vector( sum_table[metabolites,'P_GT_LT_0'] > 0.975  )
        eff_bayes = as.vector( sum_table[metabolites,'mean'] )
        
        res_bayes = compute_stats(sig_bayes,truth,'bayes', eff_bayes, true_effs )
        res = rbind(res,res_bayes,stringsAsFactors = F)
        
        res_raw = compute_stats(sig_raw,truth,'raw', eff_raw, true_effs )
        res = rbind(res,res_raw,stringsAsFactors = F)
        res_bon = compute_stats(sig_bon,truth,'bon', eff_bon, true_effs )
        res = rbind(res,res_bon,stringsAsFactors = F)
        res_bh = compute_stats(sig_bh,truth,'bh', eff_bh, true_effs )
        res = rbind(res,res_bh,stringsAsFactors = F)
        
    }
    res
}


In [32]:
apply_logistic_sim = function(met,df) {
    m = glm(died_90_day ~ I(df[,met]) ,data=df,family="binomial")
    summary(m)$coef[2,]
}

compute_stats = function(sig, truth, label, est_effs, true_effs ) {
    tp = sum(   (sig == 1) & (truth == 1)  )
    fn = sum(  (sig == 0) & (truth == 1) )
    fp = sum(  (sig == 1) & (truth == 0)   )
    tn =  sum(  (sig == 0) & (truth == 0)  )
    avg_mag_error = mean( abs(est_effs[sig & truth]/true_effs[sig & truth]) )
    num_sign_error = sum( (sign(est_effs) != sign(true_effs))[sig & truth] )
    n_sig = sum(sig)
    n_true = sum(truth)
    
    if(n_sig == 0) {
        avg_mag_error = 0
        num_sign_error = 0
    }
    return(list(label=label,tp=tp,fn=fn,fp=fp,tn=tn,avg_mag_error=avg_mag_error, num_sign_error=num_sign_error, n_sig=n_sig, n_true=n_true))
}


In [33]:
sim_data = make_df(10000,mu_0,sigma_0,10000,mu_1,sigma_1,frac_sig=1,censor=FALSE)
df_sample = sim_data$df_sample
truth = sim_data$truth
df_eff_sim = sapply(colnames(df_sample)[1:27],FUN=function( x ) { apply_logistic_sim(x,df_sample) } )
true_effs = as.numeric(df_eff_sim['Estimate',])
true_effs

[1]  0.163364533 -0.071909467  0.062469547  0.497537667  0.653125787
 [6]  0.375400572  0.228516668  0.296442698  0.248074240  0.397163651
[11]  0.007145001  0.544193714  0.560530087  0.349306688  0.055755496
[16]  0.528384923  0.195680221  0.539678610  0.491448915  0.742787623
[21]  0.260777675  0.352879483  0.484480118  0.613135152  0.257000717
[26]  0.656703858  0.170360191

In [34]:

n_sample = 100
frac_sig=0.4
max_missing=0.4
n_iter=2000
adapt_delta=0.8
max_treedepth=10
TARGET_MISSING_RATE = 0.05
beta_params = get_alpha_beta_for_target_rate(df_nmr,metabolites,TARGET_MISSING_RATE = TARGET_MISSING_RATE)
alpha = beta_params$alpha
beta = beta_params$beta
sim_number=1
res = run_sim(n_sim=1,n_0=n_sample,
              n_1=n_sample,frac_sig=frac_sig,metabolites=metabolites,
              max_missing=max_missing,iter=n_iter,sim_number=1,MISSING_RATE=TARGET_MISSING_RATE,
              alpha=alpha, beta=beta, censor=FALSE)


[1] "WRITNG OUT..."
[1] "RAW_SIM_NUMBER_1_N_SAMPLE_100_FRAC_SIG_0.4_MISSING_RATE_0.05_SIM_DATA.rds"
[1] "CONTROL"
$adapt_delta
[1] 0.8

$max_treedepth
[1] 10

[1] "NO MISING DATA SKIPPING IMPUTATION"
[1] "WRITNG OUT..."
[1] "RAW_SIM_NUMBER_1_N_SAMPLE_100_FRAC_SIG_0.4_MISSING_RATE_0.05_FIT_DATA.rds"
